In [1]:
import polars as pl
import altair as alt

/Users/sarahhussain/Documents/mscapp/fall 2025/capp30239/capp-30239-interactive/.venv/lib/python3.13/site-packages/polars/_cpu_check.py:250: RuntimeWarning: Missing required CPU features.

The following required CPU features were not detected:
    avx, avx2, fma, bmi1, bmi2, lzcnt, movbe
Continuing to use this version of Polars on this processor will likely result in a crash.
Install `polars[rtcompat]` instead of `polars` to run Polars with better compatibility.

Hint: If you are on an Apple ARM machine (e.g. M1) this is likely due to running Python under Rosetta.
It is recommended to install a native version of Python that does not run under Rosetta x86-64 emulation.

If you believe this warning to be a false positive, you can set the `POLARS_SKIP_CPU_CHECK` environment variable to bypass this check.

  warnings.warn(


In [2]:
from preprocess import clean_sector_emissions, sector_aggregates, sector_pollutant_aggregates, sector_source_pollutant_aggregates

shape: (10, 5)
┌──────────────────┬──────────────────┬───────────┬─────────────────┬──────────────────────────────┐
│ sector           ┆ source           ┆ pollutant ┆ total_emissions ┆ sector_source                │
│ ---              ┆ ---              ┆ ---       ┆ ---             ┆ ---                          │
│ str              ┆ str              ┆ str       ┆ f64             ┆ str                          │
╞══════════════════╪══════════════════╪═══════════╪═════════════════╪══════════════════════════════╡
│ Transport        ┆ Bikes            ┆ CO        ┆ 91.9247         ┆ Transport/Bikes              │
│ Transport        ┆ Rickshaws        ┆ CO        ┆ 35.7167         ┆ Transport/Rickshaws          │
│ Power Generation ┆ Fossil Fuel      ┆ SOx       ┆ 15.8765         ┆ Power Generation/Fossil Fuel │
│ Transport        ┆ Large trucks     ┆ NOx       ┆ 13.1879         ┆ Transport/Large trucks       │
│ Brick Kilns      ┆ Coal             ┆ CO        ┆ 12.7405         ┆ Brick 

In [3]:
sector_aggregates = sector_aggregates(clean_sector_emissions()).head(10)

In [4]:
sector_pollutant_aggregates = sector_pollutant_aggregates(clean_sector_emissions())

In [5]:
sector_source_pollutant_aggregates = sector_source_pollutant_aggregates(clean_sector_emissions())

In [6]:
sector_source_pollutant_aggregates.head(10)

sector,source,pollutant,total_emissions,sector_source
str,str,str,f64,str
"""Transport""","""Bikes""","""CO""",91.9247,"""Transport/Bikes"""
"""Transport""","""Rickshaws""","""CO""",35.7167,"""Transport/Rickshaws"""
"""Power Generation""","""Fossil Fuel""","""SOx""",15.8765,"""Power Generation/Fossil Fuel"""
"""Transport""","""Large trucks""","""NOx""",13.1879,"""Transport/Large trucks"""
"""Brick Kilns""","""Coal""","""CO""",12.7405,"""Brick Kilns/Coal"""
"""Transport""","""Buses""","""NOx""",12.4709,"""Transport/Buses"""
"""Domestic""","""Fuelwood & other""","""CO""",11.5929,"""Domestic/Fuelwood & other"""
"""Transport""","""Large Cars""","""CO""",11.1308,"""Transport/Large Cars"""
"""Transport""","""Large trucks""","""CO""",8.5214,"""Transport/Large trucks"""


In [7]:
sectors = ['Brick Kilns', 'Domestic', 'Industries', 'Power Generation', 'Transport']

In [9]:
source_sector_options = (
    sector_source_pollutant_aggregates
    .select(pl.col('sector_source'))
    .unique()
    .to_series()
    .to_list()
)

In [48]:
base = alt.Chart(sector_pollutant_aggregates).mark_bar().encode(
    y= alt.Y('sector:N', title = 'Sector'),
    x= alt.X('total_emissions:Q', title = 'Total Emissions (Kilotonnes)'),
    color = alt.Color('pollutant', title = 'Pollutant'),
    tooltip = ['total_emissions', 'pollutant']
)

In [49]:
# First dropdown
sector1_dropdown = alt.binding_select(options=sectors, name='Sector 1: ')
sector1_select = alt.selection_point(fields=['sector'], bind=sector1_dropdown, name="sector1")

# Second dropdown
sector2_dropdown = alt.binding_select(options=sectors, name='Sector 2: ')
sector2_select = alt.selection_point(fields=['sector'], bind=sector2_dropdown, name="sector2")


In [50]:
filter_sector = base.add_params(
    sector1_select, sector2_select).transform_filter(sector1_select | sector2_select).properties(title='Total Sectoral Emissions')

In [51]:
filter_sector

alt.Chart(...)

In [23]:
sector_source_dropdown1 = alt.binding_select(options=source_sector_options, name='Sector/Source 1')
sector_source_select1 = alt.selection_point(fields=['sector_source'], bind=sector_source_dropdown1)

sector_source_dropdown2 = alt.binding_select(options=source_sector_options, name='Sector/Source 2')
sector_source_select2 = alt.selection_point(fields=['sector_source'], bind=sector_source_dropdown2)


In [66]:
base2 = alt.Chart(sector_source_pollutant_aggregates).mark_bar().encode(
        x = alt.X('total_emissions:Q', title = 'Total Emissions (Kilotonnes)'),
        y = alt.Y('source', title = 'Source'),
        color = alt.Color('pollutant', title = 'Pollutant'),
    tooltip = ['total_emissions', 'pollutant']
    )

In [63]:
filter_sector_source = base2.add_params(
    sector1_select).transform_filter(sector1_select)

In [64]:
filter_sector_source

alt.Chart(...)

In [11]:
base3 = alt.Chart(sector_source_pollutant_aggregates).mark_bar().encode(
        x = alt.X('total_emissions:Q', title = 'Total Emissions (Kilotonnes)'),
        y = alt.Y('sector_source', title = 'Sector/Source'),
        color = alt.Color('pollutant', title = 'Pollutant'),
    tooltip = ['total_emissions', 'pollutant']
    )

In [24]:

filter_sector_source2 = base3.add_params(
    sector_source_select1, sector_source_select2).transform_filter(sector_source_select1 | sector_source_select2).properties(title='Total Sectoral Emissions')

In [25]:
filter_sector_source2

alt.Chart(...)